In [1]:
import os

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    FOR_SUBMISSION = True
else:
    FOR_SUBMISSION = False

In [2]:
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [4]:
import gc
device = 'cuda'

In [5]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_name = "/kaggle/input/deepseek-math-7b-rl/transformers/7b-rl/1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

re = naive_parse("There are 5 such numbers. So there are 5 special numbers with at most 36 digits. The answer is $\boxed{52}$.")
print(re)

52


In [8]:
import re
import sys
import subprocess


def process_output(output):
    result = output
    
    try:
        code = output.split('```')[1][7:]

        with open('code.py', 'w') as fout:
            fout.write(code)

        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1

        print('CODE RESULTS', code_output)
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1
    
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1
    
    return result_output, code_output

In [9]:
if not FOR_SUBMISSION:
    print("not submission")
    df_test = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
else:
    print("submission")
    import sys

    sys.path.append("/kaggle/input/ai-mathematical-olympiad-prize/")

    import aimo

    env = aimo.make_env()
    df_test_iter = env.iter_test()
    
    
    

not submission


In [11]:
instruction = '\nPlease reason step by step, and put your final answer within \\boxed{}.'
if FOR_SUBMISSION:
    i = 0
    for test, submission in df_test_iter:
        try:     
            problem = test["problem"]
            print(i)
            print("problem")
            print(problem)
            messages = [ {"role": "user", "content": problem + instruction} ]
            input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
            outputs = model.generate(input_tensor.to(model.device), max_new_tokens=4098)
            result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
            print(result)
            answer = naive_parse(result)
            print('answer', answer)
            answer2 = int(answer) % 1000
            print('answer2', answer2)
            submission['answer'] = answer2
            env.predict(submission)
        except:
            print('except')
            print(i)
            submission['answer'] = 0
            env.predict(submission)
else:
    for index, row in df_test.iterrows():
        problem = row["problem"]
        print("problem")
        print(index)
        print(problem)
        messages = [ {"role": "user", "content": problem + instruction} ]
        input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
        outputs = model.generate(input_tensor.to(model.device), max_new_tokens=4098)

        result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
        print(result)
        answer = naive_parse(result)
        print('answer', answer)
        answer2 = int(answer) % 1000
        print('answer2', answer2)
        
        res, code = process_output(outputs)
        print('res', res)
        print('code', code)



problem
0
Let $k, l > 0$ be parameters. The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. These points are distance 6 apart. What is the sum of the squares of the distances from $A$ and $B$ to the origin?
 The parabola $y = kx^2 - 2kx + l$ intersects the line $y = 4$ at two points $A$ and $B$. This means that the solutions to the equation $kx^2 - 2kx + l - 4 = 0$ are the $x$-coordinates of $A$ and $B$. Let's call these solutions $x_1$ and $x_2$. We are given that $|x_2 - x_1| = 6$, which means the distance between $A$ and $B$ is 6.

The sum of the squares of the distances from $A$ and $B$ to the origin is $x_1^2 + y_1^2 + x_2^2 + y_2^2$, where $y_1 = 4$ and $y_2 = 4$ because both points lie on the line $y = 4$. So we want to find $x_1^2 + y_1^2 + x_2^2 + y_2^2 = x_1^2 + 16 + x_2^2 + 16 = x_1^2 + x_2^2 + 32$.

We know that $x_1$ and $x_2$ are the solutions to the equation $kx^2 - 2kx + l - 4 = 0$. By Vieta's formulas, we have $x_1 + x_2 = \frac{2k}

OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 26.12 MiB is free. Process 3895 has 15.87 GiB memory in use. Of the allocated memory 15.43 GiB is allocated by PyTorch, and 152.44 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF